In [1]:
# %tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import data_utils
# from tensorflow.keras.models import Sequential, save_model, load_model
from keras.utils.layer_utils import print_summary
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

2022-07-31 23:45:35.390745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 23:45:35.390798: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
len(train_data[0])

218

In [4]:
train_data = data_utils.pad_sequences(train_data, MAXLEN)
test_data = data_utils.pad_sequences(test_data, MAXLEN)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [6]:
print_summary(model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
    
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 60s 92ms/step - loss: 0.4215 - acc: 0.8053 - val_loss: 0.2816 - val_acc: 0.8842
Epoch 2/10
625/625 [==============================] - 57s 91ms/step - loss: 0.2410 - acc: 0.9092 - val_loss: 0.2933 - val_acc: 0.8932
Epoch 3/10
625/625 [==============================] - 57s 91ms/step - loss: 0.1862 - acc: 0.9319 - val_loss: 0.2901 - val_acc: 0.8816
Epoch 4/10
625/625 [==============================] - 57s 92ms/step - loss: 0.1532 - acc: 0.9457 - val_loss: 0.3160 - val_acc: 0.8886
Epoch 5/10
625/625 [==============================] - 57s 92ms/step - loss: 0.1323 - acc: 0.9539 - val_loss: 0.3306 - val_acc: 0.8856
Epoch 6/10
625/625 [==============================] - 59s 95ms/step - loss: 0.1184 - acc: 0.9597 - val_loss: 0.3168 - val_acc: 0.8888
Epoch 7/10
625/625 [==============================] - 55s 88ms/step - loss: 0.1005 - acc: 0.9662 - val_loss: 0.3668 - val_acc: 0.8838
Epoch 8/10
625/625 [==============================] - 55s 88ms

In [20]:
model.save('/home/charlie-chin')

INFO:tensorflow:Assets written to: /home/charlie-chin/assets


INFO:tensorflow:Assets written to: /home/charlie-chin/assets


In [26]:
# model = keras.models.load_model('/home/charlie-chin/saved_model.pb')

# model = tf.saved_model.load('/home/charlie-chin')


In [8]:
# convert to h5

# model.save('model_name.h5')

model = keras.models.load_model('model_name.h5')

In [9]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 27s 32ms/step - loss: 0.4966 - acc: 0.8536
[0.4965744912624359, 0.8536400198936462]


In [10]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return data_utils.pad_sequences([tokens], MAXLEN)[0]

text = "That movie was just absolutely amazing, so freaking amazing"
encoded = encode_text(text)
print(encoded)


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [11]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just absolutely amazing so freaking amazing


In [12]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was terrible!"
predict(positive_review)

negative_review = "that movie really fucking sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


1/1 [==============================] - 0s 439ms/step
[0.79718405]
1/1 [==============================] - 0s 34ms/step
[0.40643787]


In [11]:
# path_to_file = tf.keras.utils.get_file('factotum.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [13]:
# Read, then decode for py2 compat.
text = open('/home/charlie-chin/factotum.txt', 'rb').read().decode(encoding='utf-8')
# Chinaski

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [3]:
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# lets look at how part of our text is encoded
print("Text:", text[:300])
# print("Encoded:", text_to_int(text[:13]))

Text: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [7]:
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [8]:
print("Encoded:", text_to_int(text[:13]))

Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [10]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

2022-07-31 23:46:05.203115: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-31 23:46:05.203175: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lenovo-C540): /proc/driver/nvidia/version does not exist


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [16]:
print(dataset.take(1))

<TakeDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


In [17]:
print(data.take(1))

<TakeDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [18]:
for input_example_batch, target_example_batch in data.take(1):
#     global example_batch_predictions
    example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


2022-07-31 23:46:11.704364: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2022-07-31 23:46:11.709493: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.


In [19]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.6581415e-03  2.3636459e-03 -3.8874033e-03 ...  1.6289169e-03
    2.9568605e-03  7.6596066e-04]
  [-1.8038116e-03 -1.4459093e-03  9.7385002e-04 ...  4.1218735e-03
   -1.5728548e-04  4.5654075e-03]
  [-4.6740384e-03  1.2562384e-03 -3.5524417e-03 ...  4.8077861e-03
    3.4265940e-03  3.9500310e-03]
  ...
  [-1.7590048e-03  1.3837002e-03 -6.8057328e-05 ... -1.0800315e-03
   -6.1123148e-03  3.1818745e-03]
  [-3.0875108e-03  3.0013628e-03 -4.9328902e-03 ...  1.6749259e-03
   -2.2890996e-03  2.7728286e-03]
  [-2.1679816e-03  2.0522121e-03 -5.1994030e-03 ...  1.0803761e-04
   -6.7133261e-03  1.3324291e-03]]

 [[ 6.5799756e-04  4.6394314e-04  9.9630211e-05 ... -2.3092623e-03
   -3.8308310e-03 -4.2605179e-04]
  [-9.1999471e-03 -9.4744074e-04 -3.1420665e-03 ... -6.1509991e-03
   -5.0447872e-03 -6.5396908e-03]
  [-5.8108978e-03  1.2550820e-03  3.0525093e-04 ... -6.0491143e-03
   -2.0101545e-03 -2.7650197e-03]
  ...
  [-3.2898542e-03 -6.7514810e-04 -1.8008612e-04 ...  4.7019143e

In [20]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-2.6581415e-03  2.3636459e-03 -3.8874033e-03 ...  1.6289169e-03
   2.9568605e-03  7.6596066e-04]
 [-1.8038116e-03 -1.4459093e-03  9.7385002e-04 ...  4.1218735e-03
  -1.5728548e-04  4.5654075e-03]
 [-4.6740384e-03  1.2562384e-03 -3.5524417e-03 ...  4.8077861e-03
   3.4265940e-03  3.9500310e-03]
 ...
 [-1.7590048e-03  1.3837002e-03 -6.8057328e-05 ... -1.0800315e-03
  -6.1123148e-03  3.1818745e-03]
 [-3.0875108e-03  3.0013628e-03 -4.9328902e-03 ...  1.6749259e-03
  -2.2890996e-03  2.7728286e-03]
 [-2.1679816e-03  2.0522121e-03 -5.1994030e-03 ...  1.0803761e-04
  -6.7133261e-03  1.3324291e-03]], shape=(100, 65), dtype=float32)


In [21]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-2.6581415e-03  2.3636459e-03 -3.8874033e-03  4.2635123e-03
 -2.6893173e-04 -1.5991735e-03 -1.9117077e-03 -8.9802977e-04
  5.0622793e-03 -2.8919349e-03 -4.1993419e-03  3.1710120e-03
 -3.0989121e-03 -1.1679454e-03 -4.4322419e-03 -2.9219007e-03
  4.1865157e-03 -2.4056223e-03  6.0539681e-04  3.3391153e-03
 -9.2532905e-04  1.7211623e-03 -1.5615821e-03 -4.3682791e-03
 -3.0893236e-03  4.0713593e-04 -6.5425062e-04  2.0946499e-03
 -4.8261764e-04 -9.8841521e-04  4.6673855e-03  5.8853757e-03
  5.0863293e-03  2.7425901e-04  2.9035306e-03 -1.9751466e-03
 -2.4468405e-05 -4.2793900e-03 -3.1805355e-03 -2.3184998e-03
 -6.3780742e-04 -5.0550350e-03  8.2702748e-04 -2.9024163e-03
 -1.1926051e-03 -4.3999441e-03  1.1473444e-03 -1.2847086e-03
 -5.6837257e-03  1.2848447e-03 -1.7764760e-03  2.5863461e-03
  3.5549630e-05 -3.8469376e-03 -4.5221578e-03  4.2800969e-03
 -2.7014217e-03 -6.8631279e-03  7.0651062e-03  3.0789394e-03
  6.1672414e-05 -1.4926331e-03  1.6289169e-03  2.9568605e-03
  7.659606

In [22]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"jPtCNAX.ms3,ZyWrtYtUpKUQ:APX\nUa: hu.KXUiW:f'EBLc$h,wdsq bgFT'v;rXHelHoKEKCaB,CTFoQlvAYmWP$KnqS?ez,Jj"

In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [24]:
# model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/home/charlie-chin/william_model/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
# history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

In [27]:
# model.save('/home/charlie-chin/william_model')

In [28]:
# model = keras.models.load_model('model_name.h5')

In [40]:
model = keras.models.load_model('/home/charlie-chin/william_model', custom_objects={'loss':loss})

In [41]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [42]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [43]:
checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./william_model/training_checkpoints/ckpt_"))
# model.load_weights(tf.train.Checkpoint("/home/charlie-chin/william_model/training_checkpoints/ckpt_" + str(checkpoint_num)+".data-00000-of-00001"))
model.load_weights("/home/charlie-chin/william_model/training_checkpoints/ckpt_" + str(checkpoint_num))



In [ ]:
os.listdir(checkpoint_dir)

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [35]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [44]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string:  juliette


julietter'd by,
I live in lives 'God send furbroa and warwick?

QUEEN MARGARET:
Let him desister; and tells him in the steel
Whis master's head,
And give his love and speechless heavior spent
That would not pey you without unknown:
Indued, sometime master, thou hast sandsto perugance

ROMEO:
Who's that a journey? for your bloody breast?
What still-meant cannot charge thee all of garteness,
And grew thy body scops: a' 'twas chief, but
most vonours that goes, and therefore fire,
And it is, crutling somewhat very oddenance
Is talk, the fire did not be seen then trage his enemies.
Alack, my Loed Gloreo, and you shall peck,
And now my lords, lift on their death in about chaste,
And brother Walting now by necesses more
Of when you having only here call it,
That you are set in standers kind, that I shoul
